In [2]:
# Import Libraries
import numpy as np
import pandas as pd
from keras.optimizers import *
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from sklearn.model_selection import StratifiedShuffleSplit
import mnist

Using TensorFlow backend.


In [3]:
#Dataset
dataset = pd.read_csv('dataset/data_windows.csv').iloc[:,1:]

In [4]:
dataset.shape

(7462, 529)

In [8]:
7462/ (16)

466.375

In [139]:
#Delete NAN values
"""
dataset_x = []
dataset_y = []
for label in range(1,17):
    data = dataset[dataset.iloc[:,0] == label]
    for index, row in data.iterrows():
        if (len(dataset_x) <= (26*label)):
            dataset_x.append(np.nan_to_num(dataset.iloc[index,1:].values))
            dataset_y.append(dataset.iloc[index,0:1].values)
        else:
            break
"""
dataset_x = []
dataset_y = []

count = np.zeros(17)

for label in range(1,17):
    data = dataset[dataset.iloc[:,0] == label]
    for index, row in data.iterrows():
        #if (len(dataset_x) <= (210*label)):
        dataset_x.append(np.nan_to_num(dataset.iloc[index,1:].values))
        dataset_y.append(dataset.iloc[index,0:1].values)
        count[label]+=1
    
dataset_x = np.array(dataset_x)
dataset_y = np.array(dataset_y)

In [140]:
dataset_x.shape,dataset_y.shape

((29848, 132), (29848, 1))

In [141]:
print(min(count[1:]))

840.0


In [142]:
#Normalice X values
mean = np.sum(dataset_x,axis=0,keepdims=True)/dataset_x.shape[0]
normal_variance = np.sum(dataset_x**2,axis=0,keepdims=True)/dataset_x.shape[0]
dataset_x_normalice = dataset_x-mean
dataset_x_normalice = dataset_x_normalice/np.sqrt(normal_variance+1e-8)

In [143]:
dataset_y_reshape = dataset_y.reshape(1,dataset_y.shape[0]).flatten().astype(int)
dataset_y_reshape[20:40]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [144]:
#Create Y binary class vector
y_train_vector = dataset_y_reshape
print(y_train_vector[0])
print(y_train_vector.shape)

1
(29848,)


In [145]:
#Dividimos en train y test
#kf = KFold(n_splits=4)
sss =  StratifiedShuffleSplit(n_splits=4,test_size=0.2, random_state=1)
#kf.get_n_splits(dataset_x_normalice)
sss.get_n_splits(dataset_x_normalice, y_train_vector)
index_kfol = []
for train_index, test_index in sss.split(dataset_x_normalice, y_train_vector):
    index_kfol.append([train_index,test_index])

In [203]:
def test_configuration(lr_v = [0.001], beta_1_v=[0.9],beta_2_v=[0.999] ,epochs_n = [10], batch_size_n = [32], input_shape_n = 132):
    
    for batch in batch_size_n:
        for epoch in epochs_n:
            for beta2 in beta_2_v:
                for beta1 in beta_1_v:
                    for lr in lr_v:
                        
                        #Dividimos en train y test
                        sss =  StratifiedShuffleSplit(n_splits=4,test_size=0.2, random_state=1)
                        #kf.get_n_splits(dataset_x_normalice)
                        sss.get_n_splits(dataset_x_normalice, y_train_vector)
                        index_kfol = []
                        for train_index, test_index in sss.split(dataset_x_normalice, y_train_vector):
                            index_kfol.append([train_index,test_index])
                        
                        #For save accuracies
                        results = {}
                        
                        # Create Model
                        model = Sequential([
                            Dense(128, activation='relu', input_shape=(input_shape_n,)),
                            Dense(64, activation='relu'),
                            Dense(32, activation='relu'),
                            Dense(16, activation='softmax'),
                        ])

                        #Compile Model
                        model.compile(
                          optimizer=Adam(learning_rate=lr, beta_1=beta1, beta_2=beta2, amsgrad=False),
                          loss='categorical_crossentropy',
                          metrics=['accuracy'],
                        )
                        
                        Wsave = model.get_weights()

                        for g in range(4):

                            model.set_weights(Wsave)

                            #Get K-fold group 
                            train_index = index_kfol[g][0]
                            test_index = index_kfol[g][1]

                            x_train = dataset_x_normalice[train_index]
                            y_train = y_train_vector[train_index]
                            x_test = dataset_x_normalice[test_index]
                            y_test = y_train_vector[test_index]  

                            history = model.fit(
                              x_train, # training data
                              to_categorical(y_train-1), # training targets
                              epochs=epoch,
                              batch_size=batch,
                              verbose=0
                            )

                            results[str(g)+"train"] = history.history['accuracy'][-1]

                            output = model.evaluate(x_test,to_categorical(y_test-1))

                            results[str(g)+"test"] = output[1]
                            
                        #Print Results
                        print("______________________Configuration______________________")
                        print("LR:"+str(lr)+"- Beta_1:"+str(beta1)+"- Beta_2:"+str(beta2))
                        print("Epochs:"+str(epoch)+"- Bach:"+str(batch)+"- Beta_2:"+str(beta2))
                        print("_________________________________________________________")
                        prom_train = []
                        prom_test = []
                        for g in range(4):
                            print("Group:"+str(g))
                            print("Train:"+str(results[str(g)+"train"]*100)+"% -Test:"+str(results[str(g)+"test"]*100)+"%")
                            prom_train.append(results[str(g)+"train"]*100)
                            prom_test.append(results[str(g)+"test"]*100)

                        print("Promedio Train")
                        print(sum(prom_train)/4)
                        print("Promedio Test")
                        print(sum(prom_test)/4)
                        print("_______________________ENDTEST_________________________")                  
                        

In [ ]:
test_configuration(lr_v=[0.1,0.01,0.001,0.0001])

5970/5970 [==============================] - 0s 10us/step


In [196]:
#Parameters and hyperparameters
input_shape_n = 132
epochs_n = 20
batch_size_n = 64

In [197]:
#For save accuracies
results = {}

In [198]:
# Create Model
model = Sequential([
    Dense(128, activation='relu', input_shape=(input_shape_n,)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='softmax'),
])

#Compile Model
model.compile(
  optimizer=Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False),
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)

In [199]:
Wsave = model.get_weights()

for g in range(4):
    
    model.set_weights(Wsave)
    
    #Get K-fold group 
    train_index = index_kfol[g][0]
    test_index = index_kfol[g][1]

    x_train = dataset_x_normalice[train_index]
    y_train = y_train_vector[train_index]
    x_test = dataset_x_normalice[test_index]
    y_test = y_train_vector[test_index]  
        
    history = model.fit(
      x_train, # training data
      to_categorical(y_train-1), # training targets
      epochs=epochs_n,
      batch_size=batch_size_n
    )
    
    results[str(g)+"train"] = history.history['accuracy'][-1]
    
    output = model.evaluate(x_test,to_categorical(y_test-1))
    
    results[str(g)+"test"] = output[1]    

Epoch 1/20
23878/23878 [==============================] - 0s 16us/step - loss: 0.8120 - accuracy: 0.7781
Epoch 2/20
23878/23878 [==============================] - 0s 13us/step - loss: 0.3254 - accuracy: 0.9058
Epoch 3/20
23878/23878 [==============================] - 0s 13us/step - loss: 0.2566 - accuracy: 0.9221
Epoch 4/20
23878/23878 [==============================] - 0s 13us/step - loss: 0.2169 - accuracy: 0.9319
Epoch 5/20
23878/23878 [==============================] - 0s 13us/step - loss: 0.1938 - accuracy: 0.9392
Epoch 6/20
23878/23878 [==============================] - 0s 13us/step - loss: 0.1659 - accuracy: 0.9468
Epoch 7/20
23878/23878 [==============================] - 0s 13us/step - loss: 0.1515 - accuracy: 0.9507
Epoch 8/20
23878/23878 [==============================] - 0s 13us/step - loss: 0.1417 - accuracy: 0.9535
Epoch 9/20
23878/23878 [==============================] - 0s 13us/step - loss: 0.1259 - accuracy: 0.9584
Epoch 10/20
23878/23878 [==============================

Epoch 17/20
23878/23878 [==============================] - 0s 13us/step - loss: 0.0810 - accuracy: 0.9723
Epoch 18/20
23878/23878 [==============================] - 0s 13us/step - loss: 0.0769 - accuracy: 0.9726
Epoch 19/20
23878/23878 [==============================] - 0s 13us/step - loss: 0.0772 - accuracy: 0.9741
Epoch 20/20
5970/5970 [==============================] - 0s 10us/step


In [200]:
#Print Results
prom_train = []
prom_test = []
for g in range(4):
    print("Group:"+str(g))
    print("Train:"+str(results[str(g)+"train"]*100)+"% -Test:"+str(results[str(g)+"test"]*100)+"%")
    prom_train.append(results[str(g)+"train"]*100)
    prom_test.append(results[str(g)+"test"]*100)

print("Promedio Train")
print(sum(prom_train)/4)
print("Promedio Test")
print(sum(prom_test)/4)

Group:0
Train:97.91858792304993% -Test:94.38861012458801%
Group:1
Train:97.77200818061829% -Test:95.1256275177002%
Group:2
Train:97.6170539855957% -Test:95.56114077568054%
Group:3
Train:97.62961864471436% -Test:95.46063542366028%
Promedio Train
97.73431718349457
Promedio Test
95.13400346040726
